In [1]:
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns

import xlrd
import xlwt

from openpyxl import load_workbook

import math
%matplotlib inline

In [2]:
movie = pd.read_csv("E:/MoiveData/movie.csv")
movie_actor = pd.read_csv("E:/MoiveData/movie_actor.csv")

In [3]:
movie.head(1)

,MovieID,MovieCName,MovieEName,MovieType,MovieLength,MovieReleaseTime,MovieStandard,MovieSumBoxOffice,MovieAvgPrice,MovieAvgPeople,MovieWomIndex
0,273,舞吧！昴,Dance Subaru,歌舞/剧情,10.0,20101203.0,2D,471000.0,NaN,NaN,NaN


In [4]:
movie_actor.head(1)

,MovieID,ActorID,ActorRank,ActorRole
0,273,2288,1,actor


In [5]:
actor = pd.read_csv("E:/MoiveData/actor.csv")
actor.head(1)

,ActorID,ActorCName,ActorEName,ActorNation
0,19,马晓晴,Xiaoqing Ma,中国


In [6]:
typeid = {} # type 的 id
movie_type = {}
movie_box = {}

N_type = 0

for _,row in movie.iterrows():
    if not isinstance(row['MovieType'], str):
        continue
    if row['MovieSumBoxOffice'] == 'NaN' or math.isnan(row['MovieSumBoxOffice']):
        continue
        
    ID = row['MovieID']
    
    # 类型编号
    types = row['MovieType'].split('/')
    for t in types:
        if t in typeid.keys():
            continue
        else:
            typeid[t] = N_type
            N_type += 1
            
    # 电影类型
    movie_type[ID] = []
    for t in types:
        movie_type[row['MovieID']].append( typeid[t] )
        
    # 电影票房
    movie_box[ID] = row['MovieSumBoxOffice']

In [7]:
idtype = {} # id的type

for key,val in typeid.items():
    idtype[val] = key

for key,val in idtype.items():
    print key,
    print val

0 歌舞
1 剧情
2 纪录片
3 传记
4 动作
5 冒险
6 青春
7 爱情
8 武侠
9 古装
10 科幻
11 奇幻
12 功夫
13 战争
14 运动
15 喜剧
16 悬疑
17 都市
18 时尚
19 职场
20 惊悚
21 动画
22 励志
23 公路
24 恐怖
25 音乐
26 怀旧
27 军旅
28 犯罪
29 儿童
30 革命
31 亲情
32 农村
33 同性
34 舞台艺术片
35 灾难
36 动物
37 灵异
38 文艺
39 心理
40 警匪
41 戏曲
42 玄幻
43 友情
44 反腐
45 真人秀
46 综艺大电影
47 枪战
48 主旋律
49 民族
50 黑帮
51 穿越
52 校园
53 军事
54 神话
55 黑色幽默


In [8]:
actor_box = {} # actor_box[actorid][typeid] -> 票房

for _,row in movie_actor.iterrows():
    
    MovieID = row['MovieID']
    ActorID = row['ActorID']
    Rank = row['ActorRank']
    if not movie_box.has_key(MovieID):
        continue

    if not actor_box.has_key(ActorID):
        actor_box[ActorID] = {}
        
        
    tem_box = actor_box[ActorID]
    for t in movie_type[MovieID]:
        if not tem_box.has_key(t):
            tem_box[t] = 0
            
        tem_box[t] += movie_box[MovieID]/Rank   # 权重 movie_box[MovieID] / Rank

In [9]:
actor_box[87]

{1: 163000.0,
 4: 50765250.0,
 7: 2808000.0,
 10: 47565000.0,
 14: 2808000.0,
 15: 50928250.0,
 28: 3200250.0}

In [10]:
type_box4act = {} # # type_box4act[typeid] -> (票房, actorid)

for act,type_box in actor_box.items():

    for t,box in type_box.items():
        if not type_box4act.has_key(t):
            type_box4act[t] = []
        type_box4act[t].append( (box , act) ) 

In [11]:
for box4act in type_box4act.values():
    box4act.sort(reverse=True)

In [12]:
type_box4act[7]

[(1737536083.3333333, 27L),
 (1589855750.0, 7415L),
 (1561541058.8235295, 2180209L),
 (1473331500.0, 10220L),
 (1441610000.0, 2190167L),
 (1408728166.6666667, 1374L),
 (1306949000.0, 4291L),
 (1201112500.0, 2175940L),
 (1170026000.0, 3341L),
 (1164516000.0, 1204L),
 (1124443500.0, 2013052L),
 (1094399000.0, 3307L),
 (1050555791.6666666, 113L),
 (1009990000.0, 1877315L),
 (998349268.1818182, 32548L),
 (986640000.0, 411L),
 (974463000.0, 8663L),
 (955447055.5555556, 1903494L),
 (919702545.4545455, 5302L),
 (877908500.0, 1903300L),
 (868013500.0, 1903012L),
 (853285500.0, 3449L),
 (809302136.8421053, 2185167L),
 (807925200.0, 1878021L),
 (803342750.0, 1877772L),
 (776861000.0, 2201375L),
 (771758258.974359, 2009905L),
 (744302000.0, 9949L),
 (744302000.0, 1168L),
 (742494250.0, 6904L),
 (720805000.0, 2204212L),
 (720805000.0, 9450L),
 (719182000.0, 1904801L),
 (702105000.0, 1948635L),
 (684081333.3333334, 1877325L),
 (663061000.0, 4177L),
 (658609000.0, 2177627L),
 (658609000.0, 2176960L)

In [ ]:
w = xlwt.Workbook()
wss = []
for i in range(55):
    ws = w.add_sheet('type'+str(i))
    wss.append(ws)

# for j in range(key):
#     Q[j] = type_box4act[j]
for k in range(55):
    Q = type_box4act[k]
    l = range(len(Q))
    ws = wss[k]

    for i in l:
        ws.write(i,0,idtype[k].decode('utf8'))     
        ws.write(i,1,Q[i][0])
        ws.write(i,2,Q[i][1])
    
w.save('E:/MoiveData/type_actor.xls') #保存